<a href="https://colab.research.google.com/github/AndrewRucker/Fantasy-Football-and-Betting/blob/main/BestBall/Underdog/Rookie-and-Sophomores/Rookie_and_Soph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Library imports and dataset creation

In [91]:
!pip install -q altair_saver

In [110]:
!apt-get -qq install chromium-chromedriver

Selecting previously unselected package chromium-codecs-ffmpeg-extra.
(Reading database ... 155514 files and directories currently installed.)
Preparing to unpack .../chromium-codecs-ffmpeg-extra_100.0.4896.127-0ubuntu0.18.04.1_amd64.deb ...
Unpacking chromium-codecs-ffmpeg-extra (100.0.4896.127-0ubuntu0.18.04.1) ...
Selecting previously unselected package chromium-browser.
Preparing to unpack .../chromium-browser_100.0.4896.127-0ubuntu0.18.04.1_amd64.deb ...
Unpacking chromium-browser (100.0.4896.127-0ubuntu0.18.04.1) ...
Selecting previously unselected package chromium-browser-l10n.
Preparing to unpack .../chromium-browser-l10n_100.0.4896.127-0ubuntu0.18.04.1_all.deb ...
Unpacking chromium-browser-l10n (100.0.4896.127-0ubuntu0.18.04.1) ...
Selecting previously unselected package chromium-chromedriver.
Preparing to unpack .../chromium-chromedriver_100.0.4896.127-0ubuntu0.18.04.1_amd64.deb ...
Unpacking chromium-chromedriver (100.0.4896.127-0ubuntu0.18.04.1) ...
Setting up chromium-cod

In [111]:
!npm install --silent vega-lite vega-cli canvas

[canvas] Success: "/content/node_modules/canvas/build/Release/canvas.node" is installed via remote
+ vega-cli@5.22.1
+ canvas@2.9.1
+ vega-lite@5.2.0
added 143 packages from 81 contributors and audited 144 packages in 13.476s

8 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



   ╭───────────────────────────────────────────────────────────────╮
   │                                                               │
   │      New major version of npm available! 6.14.8 → 8.8.0       │
   │   Changelog: https://github.com/npm/cli/releases/tag/v8.8.0   │
   │               Run npm install -g npm to update!               │
   │                                                               │
   ╰───────────────────────────────────────────────────────────────╯



In [108]:
import pandas as pd
import seaborn as sns
import numpy as np
import altair as alt
from IPython.core.pylabtools import figsize

sns.set_context(font_scale=2)

# to bypass warnings in various dataframe assignments
pd.options.mode.chained_assignment = None

In [93]:
#Read in CSV to dataframe
url = 'https://raw.githubusercontent.com/AndrewRucker/Fantasy-Football-and-Betting/main/BestBall/Underdog/Rookie-and-Sophomores/RookSophTeams.csv'
df = pd.read_csv(url, index_col='Team')
df = df.dropna(how='all')

df

,POS,Name,TEAM,Round,Pick,ADP,Value,Draft Start,Cumulative Val
Team,,,,,,,,,
1.0,W/T,Jamar Chase,CIN,1.0,1.0,1.1,-0.1,4/13/2022,5.4
1.0,RB,Breece Hall,NaN,2.0,8.0,8.2,-0.2,4/13/2022,5.4
1.0,QB,Justin Fields,CHI,3.0,9.0,7.2,1.8,4/13/2022,5.4
1.0,QB,Mac Jones,NE,4.0,16.0,14.3,1.7,4/13/2022,5.4
1.0,RB,Elijah Mitchell,SF,5.0,17.0,17.5,-0.5,4/13/2022,5.4
...,...,...,...,...,...,...,...,...,...
150.0,W/T,Rashod Bateman,BAL,8.0,29.0,25.2,3.8,4/25/2022,9.4
150.0,RB,Kenneth Gainwell,PHI,9.0,36.0,37.4,-1.4,4/25/2022,9.4
150.0,RB,Trey Sermon,SF,10.0,37.0,37.0,0.0,4/25/2022,9.4


In [94]:
#Creation of dataset with player drafted with value according to ADP by draft date
some_values = ('Kenny Pickett', 'Matt Corral', 'Malik Willis', 'Sam Howell', 'Desmond Ridder', 'Trevor Lawrence', 'Justin Fields', 'Davis Mills', 'Mac Jones', 'Zach Wilson')
df_date = df.drop(columns=['POS', 'Round', 'Pick', 'ADP', 'Cumulative Val'])
df_date = df_date.loc[df_date['Name'].isin(some_values)]
df_date


,Name,TEAM,Value,Draft Start
Team,,,,
1.0,Justin Fields,CHI,1.8,4/13/2022
1.0,Mac Jones,NE,1.7,4/13/2022
2.0,Trevor Lawrence,JAX,0.1,4/13/2022
2.0,Zach Wilson,NYJ,2.5,4/13/2022
3.0,Zach Wilson,NYJ,-2.5,4/13/2022
...,...,...,...,...
147.0,Desmond Ridder,NaN,0.0,4/25/2022
148.0,Justin Fields,CHI,-2.6,4/25/2022
148.0,Malik Willis,NaN,0.3,4/25/2022


#### Initial data manipulation

In [95]:
df1 = df.groupby('Team')['POS'].apply(lambda x: ', '.join(x.astype(str))).reset_index()

df1['RB'] = df1['POS'].str.count('RB')
df1['W/T'] = df1['POS'].str.count('W/T')
df1['QB'] = df1['POS'].str.count('QB')

df1['build'] = df1['QB'].astype(str) + "/" + df1['RB'].astype(str) + "/" + df1['W/T'].astype(str)
df1 = df1.drop(columns=['POS', 'RB', 'W/T', 'QB'])

In [96]:
df2 = pd.merge(df, df1, how='left', on = 'Team')
df3 = df2.groupby('Team')['Name'].apply(lambda x: ', '.join(x.astype(str))).reset_index()
df2 = pd.merge(df2, df3, how='left', on='Team')

In [97]:
final_df = df2.drop(columns=['POS', 'Name_x', 'TEAM', 'Round', 'Pick', 'ADP', 'Value'])
final_df = final_df.drop_duplicates()
final_df ['Players'] = final_df['Name_y']
final_df = final_df.drop(columns=['Name_y'])
final_df

split_df = pd.concat([final_df['Team'], final_df['Players'].str.split(', ', expand=True)], axis=1)
split_df.rename({0 : 1, 1 : 2, 2 : 3, 3 : 4, 4 : 5, 5 : 6, 6 : 7, 7 : 8, 8 : 9, 9 : 10, 10 : 11, 11 : 12}, axis='columns', inplace=True)

In [98]:
df1 = pd.merge(df1, df2, how='left', on = 'Team')
df1 = df1.drop(columns=['POS', 'Name_x', 'TEAM', 'Round', 'Pick', 'ADP', 'Value', 'Name_y'])
df1

,Team,build_x,Draft Start,Cumulative Val,build_y
0,1.0,2/4/6,4/13/2022,5.4,2/4/6
1,1.0,2/4/6,4/13/2022,5.4,2/4/6
2,1.0,2/4/6,4/13/2022,5.4,2/4/6
3,1.0,2/4/6,4/13/2022,5.4,2/4/6
4,1.0,2/4/6,4/13/2022,5.4,2/4/6
...,...,...,...,...,...
1795,150.0,2/4/6,4/25/2022,9.4,2/4/6
1796,150.0,2/4/6,4/25/2022,9.4,2/4/6
1797,150.0,2/4/6,4/25/2022,9.4,2/4/6
1798,150.0,2/4/6,4/25/2022,9.4,2/4/6


#### Player/Build viz and stats

In [99]:
#Exposure calc
exp_df = df2['Name_x'].value_counts()
exp_df = exp_df.reset_index()
exp_df['Count'] = exp_df['Name_x']
exp_df['%'] = round((exp_df['Count'] / df2['Team'].max()) * 100, 1)
exp_df = exp_df.drop(columns=['Name_x'])
exp_df.rename({'index' : 'Player'}, axis='columns', inplace=True)
exp_df

,Player,Count,%
0,Jahan Dotson,60,40.0
1,Rondale Moore,59,39.3
2,Treylon Burks,59,39.3
3,Drake London,57,38.0
4,Javonte Williams,51,34.0
...,...,...,...
66,Hassan Haskins,1,0.7
67,Romeo Doubs,1,0.7
68,Tyler Badie,1,0.7
69,Jerome Ford,1,0.7


In [100]:
#Player Exposure
alt.Chart(exp_df).mark_bar().encode(
    x='Count',
    y=alt.Y("Player", sort='-x'),
    tooltip=['Player','Count', '%']
).properties(height=700)

alt.Chart(...)

In [101]:
#My draft tendencies for QBs over time, by value

stripplot =  alt.Chart(df_date, width=40).mark_circle(size=8).encode(
    x=alt.X(
        'jitter:Q',
        title=None,
        axis=alt.Axis(values=[0], ticks=True, grid=False, labels=False),
        scale=alt.Scale(),
    ),
    y=alt.Y('Name'),
    tooltip=['Name', 'Draft Start', 'Value'],
    color=alt.Color('Value', scale=alt.Scale(scheme='orangered')),
    column=alt.Column(
        'Draft Start',
        header=alt.Header(
            labelAngle=-90,
            titleOrient='bottom',
            labelOrient='top',
            labelAlign='right',
            labelPadding=3,
        ),
    ),
).transform_calculate(
    # Generate Gaussian jitter with a Box-Muller transform
    jitter='sqrt(-2*log(random()))*cos(2*PI*random())'
).configure_facet(
    spacing=0
).configure_view(
    stroke=None
).configure_range(
    category={'scheme' : 'dark2'}
)

stripplot

alt.Chart(...)

In [102]:
#Team Build by draft date, with cumulative ADP value

alt.Chart(df1).mark_circle(size=40).encode(
    x='Draft Start',
    y='Cumulative Val',
    color='build_x',
    tooltip=['Draft Start','Team', 'build_x', 'Cumulative Val']
).interactive()

alt.Chart(...)

In [103]:
#Breakdown of all builds
build_count = final_df['build'].value_counts()
build_count = build_count.reset_index()
build_count['Count'] = build_count['build']
build_count['%'] = round((build_count['Count'] / df2['Team'].max()) * 100, 1)
build_count = build_count.drop(columns=['build'])
build_count.rename({'index' : 'Build'}, axis='columns', inplace=True)
build_count

,Build,Count,%
0,2/3/7,59,39.3
1,2/4/6,35,23.3
2,3/3/6,27,18.0
3,3/2/7,18,12.0
4,2/2/8,10,6.7
5,3/4/5,1,0.7
6,2/4/5,1,0.7


In [104]:
#Cumulative value description
cum_val_sum = final_df['Cumulative Val'].describe()
cum_val_sum

count    151.000000
mean       6.428477
std        8.899965
min      -15.800000
25%        1.050000
50%        6.000000
75%       11.600000
max       37.700000
Name: Cumulative Val, dtype: float64

#### Player team by round search

In [117]:
#Function that returns all teams with player passed, along with the builds used with the player
def player_team_search(players):
      base = r'^{}'
      expr = '(?=.*{})'
      base = base.format(''.join(expr.format(p) for p in players))
      #Change for number of rounds
      rounds = 12
      i = 1

      with_player = final_df.loc[final_df['Players'].str.contains(base)]
      with_player = with_player.drop_duplicates(subset=['Team'])

      #Building out round by round breakdown for teams with given player
      split_df = pd.concat([final_df['Team'], final_df['Players'].str.split(', ', expand=True)], axis=1)
      split_df.rename({0 : 1, 1 : 2, 2 : 3, 3 : 4, 4 : 5, 5 : 6, 6 : 7, 7 : 8, 8 : 9, 9 : 10, 10 : 11, 11 : 12}, axis='columns', inplace=True)
      split_df = split_df.merge(with_player, on=['Team', 'Team'], how='right')
      split_df = split_df.drop_duplicates(subset=['Team'])
      print("Round by Round selections with " + ', '.join(players) + " on the team \n")
      while i <= rounds:
        round_count = split_df[i].value_counts().rename_axis('Player').reset_index(name='Times Selected')
        print("===============================================================")
        print("Round " + str(i) + " \n")
        chart = bars = alt.Chart(round_count).mark_bar().encode(
            x='Times Selected',
            y='Player'
        )
        chart.save("Page/round" + str(i) + ".html")
        i=i+1





      #List of teams with given player on roster
      #print("===========================================================")
      #print("Full Teams with " + ', '.join(players) + " on the roster \n")
      #print(with_player['Players'].value_counts())

#### Build Count with Given Players

In [119]:
#Build Count with given players function
def builds_with_players(players):
      base = r'^{}'
      expr = '(?=.*{})'
      base = base.format(''.join(expr.format(p) for p in players))
      #Change for number of rounds
      rounds = 12
      i = 1

      with_player = final_df.loc[final_df['Players'].str.contains(base)]
      with_player = with_player.drop_duplicates(subset=['Team'])    
      print("Structural builds with " + ', '.join(players) + " on the roster")
      chart = (alt.Chart(with_player).mark_circle(size=30).encode(
      x='Draft Start',
      y='Cumulative Val',
      color='build',
      tooltip=['Draft Start','Team', 'build', 'Cumulative Val']
      ).interactive())
      chart.save('Page/builds.html')
      print("\n")

##Players Drafted Entry

In [120]:
players_to_search = ['Javonte Williams', 'Breece Hall']
build = builds_with_players(players_to_search)
player_team_search(players_to_search)

Structural builds with Javonte Williams, Breece Hall on the roster


Round by Round selections with Javonte Williams, Breece Hall on the team 

Round 1 

Round 2 

Round 3 

Round 4 

Round 5 

Round 6 

Round 7 

Round 8 

Round 9 

Round 10 

Round 11 

Round 12 

